# 🌽 Corn Disease Classification - Pipeline Completo

**Pipeline de entrenamiento con MobileNetV3Large**

- **Paso 4:** Configuración de entrenamiento
- **Paso 5:** Creación del modelo
- **Paso 6:** Entrenamiento
- **Paso 7:** Evaluación y exportación con MLflow

---

**GPU Recomendada:** L4 o superior  
**Tiempo estimado:** 30-60 minutos (depende de GPU y épocas)

---

## 📦 Sección 1: Configuración Inicial y Verificación de GPU

In [ ]:
# Verificar GPU disponible
import tensorflow as tf
import sys

print("=" * 80)
print("VERIFICACIÓN DE ENTORNO")
print("=" * 80)

print(f"\nPython version: {sys.version.split()[0]}")
print(f"TensorFlow version: {tf.__version__}")

# Verificar GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\n✅ GPU detectada: {len(gpus)} dispositivo(s)")
    for gpu in gpus:
        print(f"   {gpu}")
    
    # Configurar crecimiento de memoria
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("   Memory growth habilitado")
else:
    print("\n⚠️  NO se detectó GPU - El entrenamiento será MUY lento en CPU")
    print("   Recomendación: Runtime > Change runtime type > GPU (L4)")

print("\n" + "=" * 80)

## 📂 Sección 2: Instalación de Dependencias

In [ ]:
# Instalar dependencias necesarias
print("Instalando dependencias...\n")

!pip install -q mlflow
!pip install -q seaborn

print("\n✅ Dependencias instaladas correctamente")

# Verificar versiones
import mlflow
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

print(f"\nMLflow version: {mlflow.__version__}")
print(f"Seaborn version: {sns.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 📥 Sección 3: Carga del Dataset desde Google Drive y Scripts desde GitHub

**Estrategia:**
1. **Dataset (`data_augmented/`):** Montado desde Google Drive
2. **Scripts (`src/`):** Clonados desde GitHub (repositorio `cornIA`)

**Estructura esperada:**
```
Google Drive: /Mi unidad/data_augmented/
├── train/ (916 imgs x 4 clases)
├── val/   (634 imgs total)
└── test/  (634 imgs total)

GitHub: https://github.com/afelipfo/cornIA
└── src/
    ├── training_config.py
    ├── model_creation.py
    ├── train_model.py
    └── evaluate_and_export.py
```

In [ ]:
# Paso 1: Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive montado")

# Paso 2: Clonar repositorio de GitHub
print("\n📥 Clonando repositorio de GitHub...")
!git clone https://github.com/afelipfo/cornIA.git

print("\n✅ Repositorio clonado")

# Paso 3: Copiar scripts a directorio de trabajo
import shutil
import os

if os.path.exists('cornIA/src'):
    if os.path.exists('src'):
        shutil.rmtree('src')
    shutil.copytree('cornIA/src', 'src')
    print("✅ Scripts copiados desde GitHub a ./src")
else:
    print("⚠️  No se encontró carpeta src/ en el repositorio")

In [ ]:
# Paso 4: Crear enlace simbólico a data_augmented desde Google Drive
print("📂 Configurando acceso a dataset...\n")

# Ruta del dataset en Google Drive (AJUSTA ESTA RUTA SEGÚN TU ESTRUCTURA)
drive_data_path = '/content/drive/MyDrive/data_augmented'

# Verificar si existe
if os.path.exists(drive_data_path):
    # Crear enlace simbólico
    if os.path.exists('data_augmented'):
        os.remove('data_augmented')
    os.symlink(drive_data_path, 'data_augmented')
    print(f"✅ Dataset enlazado desde: {drive_data_path}")
else:
    print(f"❌ NO ENCONTRADO: {drive_data_path}")
    print("\n⚠️  ACCIÓN REQUERIDA:")
    print("   1. Verifica que subiste data_augmented/ a Google Drive")
    print("   2. Ajusta la ruta 'drive_data_path' en esta celda")
    print("   Ejemplo: '/content/drive/MyDrive/MiCarpeta/data_augmented'")

# Paso 5: Verificar estructura completa
print("\n" + "=" * 80)
print("VERIFICANDO ESTRUCTURA DE ARCHIVOS")
print("=" * 80 + "\n")

required_structure = {
    'Carpetas de datos': [
        'data_augmented/train/Blight',
        'data_augmented/train/CommonRust',
        'data_augmented/train/GrayLeafSpot',
        'data_augmented/train/Healthy',
        'data_augmented/val',
        'data_augmented/test'
    ],
    'Scripts de entrenamiento': [
        'src/training_config.py',
        'src/model_creation.py',
        'src/train_model.py',
        'src/evaluate_and_export.py'
    ]
}

all_ok = True
for category, paths in required_structure.items():
    print(f"\n{category}:")
    for path in paths:
        if os.path.exists(path):
            print(f"  ✅ {path}")
        else:
            print(f"  ❌ {path} - NO ENCONTRADO")
            all_ok = False

if all_ok:
    print("\n" + "=" * 80)
    print("🎉 VERIFICACIÓN EXITOSA - Todos los archivos están listos")
    print("=" * 80)
else:
    print("\n" + "=" * 80)
    print("⚠️  VERIFICACIÓN FALLIDA - Corrige los errores antes de continuar")
    print("=" * 80)

## 🔧 Sección 4: Configuración de Parámetros del Pipeline

In [ ]:
# Configuración del pipeline
RANDOM_SEED = 42
EPOCHS = 50
BATCH_SIZE = 32

print("=" * 80)
print("CONFIGURACIÓN DEL PIPELINE")
print("=" * 80)
print(f"\nRandom seed: {RANDOM_SEED}")
print(f"Épocas: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print("\n" + "=" * 80)

# Agregar src al path para imports
import sys
sys.path.insert(0, './src')

print("\n✅ Parámetros configurados")
print("✅ Módulos de src/ agregados al path")

## 🎯 Sección 5: PASO 4 - Configuración de Entrenamiento

Configura:
- Data augmentation online
- Generadores de datos (train/val/test)
- Callbacks (EarlyStopping, ReduceLROnPlateau, etc.)

In [ ]:
from training_config import TrainingConfiguration
from datetime import datetime

print("\n" + "▓" * 100)
print("▓ PASO 4: CONFIGURACIÓN DE ENTRENAMIENTO")
print("▓ Configura augmentation online, generadores de datos y callbacks")
print("▓" * 100 + "\n")

try:
    # Inicializar configuración
    print("🔧 Inicializando TrainingConfiguration...\n")
    config = TrainingConfiguration(
        data_dir='./data_augmented',
        output_dir='./training_output',
        random_seed=RANDOM_SEED
    )
    
    # Ejecutar configuración completa
    print("🚀 Ejecutando configuración completa...\n")
    config.run()
    
    print("\n" + "░" * 100)
    print("░ ✅ PASO 4 COMPLETADO: Configuración de Entrenamiento")
    print("░" * 100 + "\n")
    
except Exception as e:
    print("\n" + "░" * 100)
    print("░ ❌ PASO 4 FALLÓ: Configuración de Entrenamiento")
    print("░" * 100)
    print(f"\nError: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

## 🤖 Sección 6: PASO 5 - Creación del Modelo

Crea el modelo:
- MobileNetV3Large (ImageNet)
- BatchNormalization
- Dropout(0.5)
- Dense(4, softmax)

In [ ]:
from model_creation import ModelCreator

print("\n" + "▓" * 100)
print("▓ PASO 5: CREACIÓN DEL MODELO")
print("▓ Crea MobileNetV3Large con BatchNorm, Dropout y capa de salida")
print("▓" * 100 + "\n")

try:
    # Inicializar creador de modelo
    print("🤖 Inicializando ModelCreator...\n")
    creator = ModelCreator(
        config_path='./training_output/training_config.json',
        output_dir='./model_output',
        random_seed=RANDOM_SEED
    )
    
    # Crear y compilar modelo
    print("🏗️  Ejecutando creación y compilación del modelo...\n")
    model = creator.run()
    
    print("\n" + "░" * 100)
    print("░ ✅ PASO 5 COMPLETADO: Creación del Modelo")
    print("░" * 100 + "\n")
    
except Exception as e:
    print("\n" + "░" * 100)
    print("░ ❌ PASO 5 FALLÓ: Creación del Modelo")
    print("░" * 100)
    print(f"\nError: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

## 🏋️ Sección 7: PASO 6 - Entrenamiento

Entrena el modelo:
- Con augmentation online
- Callbacks automáticos
- Métricas avanzadas (Precision, Recall, AUC)
- Gráficas automáticas

**⚠️ ADVERTENCIA:** Esta celda puede tardar 30-60 minutos dependiendo de la GPU.

In [ ]:
from train_model import ModelTrainer

print("\n" + "▓" * 100)
print("▓ PASO 6: ENTRENAMIENTO DEL MODELO")
print("▓ Entrena el modelo con augmentation, callbacks y métricas avanzadas")
print("▓" * 100 + "\n")

try:
    # Inicializar entrenador
    print("🚀 Inicializando ModelTrainer...\n")
    trainer = ModelTrainer(
        output_dir='./training_results',
        random_seed=RANDOM_SEED,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE
    )
    
    # Ejecutar entrenamiento completo
    print("🏋️  Ejecutando entrenamiento completo...\n")
    print("⏰ Esto puede tardar 30-60 minutos...\n")
    
    trainer.run()
    
    print("\n" + "░" * 100)
    print("░ ✅ PASO 6 COMPLETADO: Entrenamiento del Modelo")
    print("░" * 100 + "\n")
    
except Exception as e:
    print("\n" + "░" * 100)
    print("░ ❌ PASO 6 FALLÓ: Entrenamiento del Modelo")
    print("░" * 100)
    print(f"\nError: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

## 📊 Sección 8: Visualización de Resultados de Entrenamiento

Visualiza las gráficas generadas durante el entrenamiento.

In [ ]:
from IPython.display import Image, display
import os

print("=" * 80)
print("GRÁFICAS DE ENTRENAMIENTO")
print("=" * 80 + "\n")

# Mostrar gráfica de accuracy y loss
if os.path.exists('./training_results/training_history.png'):
    print("📈 Accuracy y Loss vs Epochs:\n")
    display(Image('./training_results/training_history.png'))
else:
    print("⚠️  No se encontró training_history.png")

# Mostrar gráfica de métricas avanzadas
if os.path.exists('./training_results/advanced_metrics.png'):
    print("\n📊 Métricas Avanzadas (Precision, Recall, AUC):\n")
    display(Image('./training_results/advanced_metrics.png'))
else:
    print("\nℹ️  No se encontró advanced_metrics.png")

## 🔬 Sección 9: PASO 7 - Evaluación y Exportación con MLflow

Evalúa el modelo en test set:
- Matriz de confusión
- Métricas completas (Accuracy, Precision, Recall, F1)
- Exportación del modelo
- Registro en MLflow

In [ ]:
from evaluate_and_export import ModelEvaluator

print("\n" + "▓" * 100)
print("▓ PASO 7: EVALUACIÓN Y EXPORTACIÓN CON MLFLOW")
print("▓ Evalúa en test, genera métricas, matriz de confusión y registra en MLflow")
print("▓" * 100 + "\n")

try:
    # Inicializar evaluador
    print("🔬 Inicializando ModelEvaluator...\n")
    evaluator = ModelEvaluator(
        model_path='./training_output/best_model.keras',
        output_dir='./evaluation_results',
        random_seed=RANDOM_SEED
    )
    
    # Ejecutar evaluación completa
    print("📊 Ejecutando evaluación y exportación completa...\n")
    evaluator.run()
    
    print("\n" + "░" * 100)
    print("░ ✅ PASO 7 COMPLETADO: Evaluación y Exportación")
    print("░" * 100 + "\n")
    
except Exception as e:
    print("\n" + "░" * 100)
    print("░ ❌ PASO 7 FALLÓ: Evaluación y Exportación")
    print("░" * 100)
    print(f"\nError: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

## 📊 Sección 10: Visualización de Resultados de Evaluación

Visualiza matriz de confusión y métricas finales.

In [ ]:
from IPython.display import Image, display
import json
import pandas as pd

print("=" * 80)
print("RESULTADOS DE EVALUACIÓN EN TEST SET")
print("=" * 80 + "\n")

# Mostrar matriz de confusión
if os.path.exists('./evaluation_results/confusion_matrix.png'):
    print("📊 Matriz de Confusión:\n")
    display(Image('./evaluation_results/confusion_matrix.png'))
else:
    print("⚠️  No se encontró confusion_matrix.png")

# Mostrar métricas finales
if os.path.exists('./evaluation_results/evaluation_results.json'):
    print("\n" + "=" * 80)
    print("📈 MÉTRICAS FINALES EN TEST SET")
    print("=" * 80 + "\n")
    
    with open('./evaluation_results/evaluation_results.json', 'r') as f:
        results = json.load(f)
    
    metrics = results['test_metrics']
    print(f"Accuracy:         {metrics['accuracy']:.4f}")
    print(f"Precision (macro): {metrics['precision_macro']:.4f}")
    print(f"Recall (macro):    {metrics['recall_macro']:.4f}")
    print(f"F1-score (macro):  {metrics['f1_macro']:.4f}")
    
    print("\n" + "-" * 80)
    print("📊 MÉTRICAS POR CLASE")
    print("-" * 80 + "\n")
    
    for class_name, class_metrics in results['class_metrics'].items():
        print(f"{class_name:15s}: Precision={class_metrics['precision']:.4f}, "
              f"Recall={class_metrics['recall']:.4f}, "
              f"F1={class_metrics['f1_score']:.4f}")
else:
    print("⚠️  No se encontró evaluation_results.json")

# Mostrar classification report
if os.path.exists('./evaluation_results/classification_report.csv'):
    print("\n" + "=" * 80)
    print("📄 CLASSIFICATION REPORT")
    print("=" * 80 + "\n")
    
    report_df = pd.read_csv('./evaluation_results/classification_report.csv', index_col=0)
    print(report_df.to_string())

## 🎉 Sección 11: Resumen Final del Pipeline

In [ ]:
print("\n" + "=" * 100)
print("█" * 100)
print("█" + " " * 35 + "PIPELINE COMPLETADO EXITOSAMENTE" + " " * 32 + "█")
print("█" * 100)
print("=" * 100)

print("\n📋 ESTADO DE PASOS:")
print("   ✅ Paso 4: Configuración de entrenamiento")
print("   ✅ Paso 5: Creación del modelo")
print("   ✅ Paso 6: Entrenamiento")
print("   ✅ Paso 7: Evaluación y MLflow")

print("\n📁 ARCHIVOS GENERADOS:")
print("   📂 training_output/")
print("      ├── training_config.json       (Configuración)")
print("      ├── best_model.keras           (Mejor modelo)")
print("      └── training_history.csv       (Historial)")
print("\n   📂 model_output/")
print("      ├── model_config.json          (Configuración del modelo)")
print("      └── model_summary.txt          (Resumen)")
print("\n   📂 training_results/")
print("      ├── training_info.json         (Info completa)")
print("      ├── training_history.png       (Gráficas)")
print("      └── advanced_metrics.png       (Métricas)")
print("\n   📂 evaluation_results/")
print("      ├── best_model.keras           (Modelo exportado)")
print("      ├── confusion_matrix.png       (Matriz)")
print("      └── classification_report.csv  (Reporte)")

print("\n" + "=" * 100)
print("\n✅ ¡Entrenamiento completado!")
print("\n📥 Para descargar los resultados:")
print("   - Haz clic derecho en los archivos/carpetas en el panel izquierdo")
print("   - Selecciona 'Download'")
print("\n🔬 MLflow:")
print("   - Todos los parámetros, métricas y artefactos registrados")
print("   - Experiment: Maize-Disease-Classification")
print("\n" + "=" * 100)

## 📥 Sección 12: Descargar Resultados (Opcional)

Descarga todos los resultados a tu máquina local.

In [ ]:
# Crear archivo ZIP con todos los resultados
import shutil
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_name = f'corn_disease_results_{timestamp}'

print(f"Creando archivo ZIP: {zip_name}.zip...\n")

# Crear ZIP con todos los directorios de resultados
shutil.make_archive(zip_name, 'zip', '.', 
                   base_dir=None,
                   root_dir='.')

# Incluir solo carpetas de resultados
folders_to_zip = [
    'training_output',
    'model_output',
    'training_results',
    'evaluation_results'
]

# Crear un ZIP más específico
!zip -r {zip_name}.zip {' '.join(folders_to_zip)} -x "*.pyc" "__pycache__/*"

print(f"\n✅ Archivo creado: {zip_name}.zip")
print(f"\n📥 Para descargar:")
print(f"   from google.colab import files")
print(f"   files.download('{zip_name}.zip')")

# Descarga automática (descomenta si quieres descarga automática)
# from google.colab import files
# files.download(f'{zip_name}.zip')

## 🔄 Sección 13: Guardar en Google Drive (Opcional)

Guarda los resultados directamente en tu Google Drive.

In [ ]:
# Copiar resultados a Google Drive
import shutil
import os

# Definir carpeta destino en Drive
drive_results_dir = '/content/drive/MyDrive/Corn_Disease_Results'

print(f"Copiando resultados a Google Drive...\n")
print(f"Destino: {drive_results_dir}\n")

# Crear carpeta en Drive si no existe
os.makedirs(drive_results_dir, exist_ok=True)

# Copiar cada carpeta de resultados
folders_to_copy = [
    'training_output',
    'model_output',
    'training_results',
    'evaluation_results'
]

for folder in folders_to_copy:
    if os.path.exists(folder):
        dest = os.path.join(drive_results_dir, folder)
        if os.path.exists(dest):
            shutil.rmtree(dest)
        shutil.copytree(folder, dest)
        print(f"✅ Copiado: {folder} → {dest}")
    else:
        print(f"⚠️  No encontrado: {folder}")

print(f"\n✅ Resultados guardados en Google Drive")
print(f"   Ubicación: {drive_results_dir}")